<a href="https://colab.research.google.com/github/Gowtham933/Supply-chain-Data-Analysis-Project/blob/main/Pyspark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cb1f44bf7e0af20a8afbb275fad62c324e6c248fe3bee18b0fd7a5ef967b2399
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
import pandas as pd
import os

In [48]:
spark = SparkSession.builder \
    .appName("Spark SQL Database") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [49]:
df = pd.read_csv('/content/supply_chain_data.csv')

In [46]:
schema = StructType([
    StructField("Product type", StringType(), True),
    StructField("SKU", StringType(), True),
    StructField("Price", FloatType(), True),
    StructField("Availability", IntegerType(), True),
    StructField("Number of products sold", IntegerType(), True),
    StructField("Revenue generated", FloatType(), True),
    StructField("Customer demographics", StringType(), True),
    StructField("Stock levels", IntegerType(), True),
    StructField("Lead times", IntegerType(), True),
    StructField("Order quantities", IntegerType(), True),
    StructField("Shipping times", IntegerType(), True),
    StructField("Shipping carriers", StringType(), True),
    StructField("Shipping costs", FloatType(), True),
    StructField("Supplier name", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Lead time", IntegerType(), True),
    StructField("Production volumes", IntegerType(), True),
    StructField("Manufacturing lead time", IntegerType(), True),
    StructField("Manufacturing costs", FloatType(), True),
    StructField("Inspection results", StringType(), True),
    StructField("Defect rates", FloatType(), True),
    StructField("Transportation modes", StringType(), True),
    StructField("Routes", StringType(), True),
    StructField("Costs", FloatType(), True)
])


In [50]:
spark_df = spark.createDataFrame(df, schema=schema)

In [51]:
spark_df.createOrReplaceTempView("Supplychain_data")

In [52]:
result = spark.sql("SELECT * FROM Supplychain_data")
result.show()

+------------+-----+---------+------------+-----------------------+-----------------+---------------------+------------+----------+----------------+--------------+-----------------+--------------+-------------+---------+---------+------------------+-----------------------+-------------------+------------------+------------+--------------------+-------+---------+
|Product type|  SKU|    Price|Availability|Number of products sold|Revenue generated|Customer demographics|Stock levels|Lead times|Order quantities|Shipping times|Shipping carriers|Shipping costs|Supplier name| Location|Lead time|Production volumes|Manufacturing lead time|Manufacturing costs|Inspection results|Defect rates|Transportation modes| Routes|    Costs|
+------------+-----+---------+------------+-----------------------+-----------------+---------------------+------------+----------+----------------+--------------+-----------------+--------------+-------------+---------+---------+------------------+---------------------

In [56]:
output_path = "/content/Supplychain_data.parquet"
spark_df.write.parquet(output_path)

In [57]:
# List files to ensure the Parquet file is saved
os.listdir('/content')

['.config',
 '.ipynb_checkpoints',
 'supply_chain_data.csv',
 'spark-warehouse',
 'Supplychain_data.parquet',
 'sample_data']

In [58]:
from google.colab import files

files.download('/content/Supplychain_data.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
# Path to the saved Parquet file
parquet_file_path = "/content/Supplychain_data.parquet"

# Load the Parquet file into a DataFrame
parquet_df = spark.read.parquet(parquet_file_path)

In [68]:
# Perform a simple query
result = spark.sql("SELECT * FROM Supplychain_data")
result.show()


+------------+-----+---------+------------+-----------------------+-----------------+---------------------+------------+----------+----------------+--------------+-----------------+--------------+-------------+---------+---------+------------------+-----------------------+-------------------+------------------+------------+--------------------+-------+----------+
|Product type|  SKU|    Price|Availability|Number of products sold|Revenue generated|Customer demographics|Stock levels|Lead times|Order quantities|Shipping times|Shipping carriers|Shipping costs|Supplier name| Location|Lead time|Production volumes|Manufacturing lead time|Manufacturing costs|Inspection results|Defect rates|Transportation modes| Routes|     Costs|
+------------+-----+---------+------------+-----------------------+-----------------+---------------------+------------+----------+----------------+--------------+-----------------+--------------+-------------+---------+---------+------------------+-------------------

In [69]:
# Example of an advanced query: Calculate average price by product type
Total_price_by_product_type = spark.sql("""
SELECT `Product type`, SUM(Price) AS Total_price
FROM Supplychain_data
GROUP BY `Product type`
""")
Total_price_by_product_type.show()

+------------+------------------+
|Product type|       Total_price|
+------------+------------------+
|    skincare|1890.3731541633606|
|   cosmetics|1491.3875029087067|
|    haircare|1564.4854910373688|
+------------+------------------+

